## Profile Score Calculation

In [0]:
from pyspark.sql.types import *
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
import pandas as pd
import numpy as np
from pyspark.sql.window import Window
from datetime import datetime
import re

spark = SparkSession.builder.getOrCreate()

In [0]:
profiles = spark.read.parquet('/dbfs/linkedin_people_train_data')
companies = spark.read.parquet('/dbfs/linkedin_train_data')
classified_jobs = spark.read.parquet("/Workspace/Users/lihi.kaspi@campus.technion.ac.il/classified_jobs.parquet")
metajob_counts = spark.read.csv("dbfs:/FileStore/tables/lihi_kaspi/counts.csv", header=True)

In [0]:
metajob_counts = metajob_counts.withColumn('Count', f.regexp_replace('Count', ',', '')) \
    .withColumn('Count', f.col('Count').cast('int')) \
    .withColumnRenamed('Job', 'metajob')

### Find Metajob and Popularity

In [0]:
jobs = profiles.select('id', f.col('experience')[0].getField('title').alias('job_title'), 'position')

# prepare the column
jobs = jobs.withColumn(
    'processed_title',
    f.when(f.col('job_title').isNotNull(), f.lower(f.col('job_title')))
    .otherwise(f.lower(f.col('position')))
)

jobs = jobs.join(classified_jobs, jobs.processed_title == classified_jobs.job_title, how='left') \
            .select('id', f.col('closest_centroid').alias('metajob'))

jobs = jobs.withColumn('metajob', f.when(f.col('metajob').isNull(), 'Miscellaneous').otherwise(f.col('metajob')))

In [0]:
profiles_df = profiles.join(jobs, on='id', how='left')
profiles_df = profiles_df.withColumn('State', f.split(f.col('city'), ', ')[1])

profiles_df = profiles_df.join(metajob_counts, on=['State', 'metajob'], how='left')
profiles_df = profiles_df.withColumn('Count', f.when(f.col('Count').isNull(), 1).otherwise(f.col('Count') + 1))

profiles_df = profiles_df.withColumn('log_count_jobs', f.log2(f.col('Count')) + 1)

In [0]:
avg_job_count = profiles_df.select('State', 'metajob', 'log_count_jobs').groupBy('State', 'metajob').avg() \
    .withColumnRenamed('avg(log_count_jobs)', 'avg_log_count_jobs')
profiles_df = profiles_df.join(avg_job_count, on=['State', 'metajob'], how='left').dropna(subset=['avg_log_count_jobs'])

### User Profile Score

In [0]:
# columns that should be filled for better understaning the user
important_cols = ['about', 'city', 'country_code', 'current_company', 'education', 'experience', 'followers', 'id', 'languages', 'name', 'position', 'posts', 'recommendations_count']

In [0]:
# find the percent of filled column out of the important ones
profiles_df = profiles_df.withColumn(
    "filled_percent",
    f.round(
        (sum(
            f.when(
                f.col(col).isNotNull() & (~f.col(col).cast("string").rlike("^\\[\\]$")), 1
            ).otherwise(0)
            for col in important_cols
        )
    ) / len(important_cols) * 100, 2))

In [0]:
# words that indicate managment/board positions
important_titles = ['president', 'ceo', 'coo', 'cfo', 'cto', 'cmo', 'cdo', 'cso','cio', 'cpo', 'cro', 'vp', 'svp', 'rvp', 'evp', 'avp', 'chief', 'executive', 'exec' 'vice president', 'director', 'partner', 'dean', 'senior', 'head', 'principal', 'provost', 'treasurer', 'chair', 'chairman', 'chairwoman', 'chancellor', 'board']

In [0]:
# count how many indicative words each user has
regex_pattern = f"({'|'.join(map(re.escape, important_titles))})"

# count word in 'positon' field
profiles_df = profiles_df.withColumn(
  'position_count', 
      f.size(
        f.expr(
          f"filter(transform(split(lower(position), ' '), word -> word RLIKE '{regex_pattern}'), x -> x)")))

# count words in 'job title' field
profiles_df = profiles_df.withColumn(
  'title_count', 
      f.size(
        f.expr(
          f"filter(transform(split(lower(experience[0].title), ' '), word -> word RLIKE '{regex_pattern}'), x -> x)")))

In [0]:
# change the range of the count to be from 1 for the log function
profiles_df = profiles_df.withColumn(
  'position_count', 
    f.when(
      f.col('position_count') < 1, 1
    ).otherwise(f.col('position_count') + 1))

profiles_df = profiles_df.withColumn(
  'title_count', 
    f.when(
      f.col('title_count') < 1, 1
    ).otherwise(f.col('title_count') + 1))

# find the avg count of the indicative words
profiles_df = profiles_df.withColumn('avg_important_count', (f.col('position_count') + f.col('title_count') + 2) / 2)

In [0]:
profiles_df = profiles_df.withColumn('avg_count', (f.col('avg_important_count') + f.col('avg_log_count_jobs')) / 2)

In [0]:
# calculate the followers score and log followers score
# profiles_df = profiles_df.withColumn('followers_score', (f.col('followers') / f.col('avg_important_count')) + 1)
profiles_df = profiles_df.withColumn('followers_score', (f.col('followers') / f.col('avg_count')) + 1)
profiles_df = profiles_df.withColumn('log_followers_score', f.log(f.col('followers_score')))

# calculate the profile score
# profiles_df = profiles_df.withColumn('profile_score', f.col('log_followers_score') * f.col('filled_percent') * f.col('avg_log_count_jobs') / 100)
profiles_df = profiles_df.withColumn('profile_score', f.col('log_followers_score') * f.col('filled_percent') / 20)

In [0]:
final_profiles_df = profiles_df.select([col for col in profiles.columns] + ['profile_score'])

In [0]:
final_profiles_df.write.mode("overwrite").parquet("/Workspace/Users/lihi.kaspi@campus.technion.ac.il/user_profiles_with_scores.parquet")

In [0]:
display(profiles_df.select('name', 'State', 'metajob', 'log_followers_score', 'avg_log_count_jobs', 'filled_percent', 'profile_score').orderBy('profile_score', ascending=False))

In [0]:
display(profiles_df.select('name', 'State', 'metajob', 'log_followers_score', 'avg_log_count_jobs', 'filled_percent', 'profile_score').orderBy('profile_score', ascending=False))

### Company Profile Score

In [0]:
companies.display()

In [0]:
companies.printSchema()

In [0]:
important_cols = ['about', 'company_size', 'country_code', 'employees_in_linkedin', 'followers', 'locations', 'headquarters', 'id', 'industries', 'name', 'organization_type', 'slogan', 'sphere', 'type', 'website']

In [0]:
# find the percent of filled column out of the important ones
companies_df = companies.withColumn(
    "filled_percent",
    f.round(
        (sum(
            f.when(
                f.col(col).isNotNull() & (~f.col(col).cast("string").rlike("^\\[\\]$")), 1
            ).otherwise(0)
            for col in important_cols
        )
    ) / len(important_cols) * 100, 2))

In [0]:
companies_df = companies_df.withColumn('locations_exploded', f.explode('locations'))

In [0]:
display(companies_df)

In [0]:
# add popularity for all cities the companies has offices in instead of important words count

## score distribution

In [0]:
import matplotlib.pyplot as plt

sample = profiles_df.select('profile_score').toPandas()

plt.figure(figsize=(10, 6))
plt.hist(sample['profile_score'], bins=30, edgecolor='k', alpha=0.7)
plt.title('Histogram of Profile Scores')
plt.xlabel('Profile Score')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()